<a href="https://colab.research.google.com/github/Vishy-A/MachineLearningIntro/blob/main/HW6/4105HW6P1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [455]:
#Vishy Adusumilli
#801185663
#HW Problem 1

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from collections import OrderedDict
from google.colab import drive


drive.mount('/content/drive')
file_path = '/content/drive/My Drive/Machine Learning/Datasets/Housing.csv'
dataset = pd.DataFrame(pd.read_csv(file_path))
dataset.head()



Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished


In [456]:
varsall = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'parking', 'prefarea', 'furnishingstatus', 'price']
varstrings = ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea', 'furnishingstatus']

def binarymap(inp):
  return inp.map({'yes' : 1, 'no' : 0, 'furnished' : 2, 'semi-furnished' : 1, 'unfurnished' : 0 })

dataset[varstrings] = dataset[varstrings].apply(binarymap)


dataset.head()


,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,1,0,0,0,1,2,1,2
1,12250000,8960,4,4,4,1,0,0,0,1,3,0,2
2,12250000,9960,3,2,2,1,0,1,0,0,2,1,1
3,12215000,7500,4,2,2,1,0,1,0,1,3,1,2
4,11410000,7420,4,1,2,1,1,1,0,1,2,0,2


In [457]:
standardScaler = StandardScaler()
minMaxScaler = MinMaxScaler()

dataset[varsall] = minMaxScaler.fit_transform(dataset[varsall])
np.random.seed(0)
dataset.head()

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,1.000000,0.396564,0.6,0.333333,0.666667,1.0,0.0,0.0,0.0,1.0,0.666667,1.0,1.0
1,0.909091,0.502405,0.6,1.000000,1.000000,1.0,0.0,0.0,0.0,1.0,1.000000,0.0,1.0
2,0.909091,0.571134,0.4,0.333333,0.333333,1.0,0.0,1.0,0.0,0.0,0.666667,1.0,0.5
3,0.906061,0.402062,0.6,0.333333,0.333333,1.0,0.0,1.0,0.0,1.0,1.000000,1.0,1.0
4,0.836364,0.396564,0.6,0.000000,0.333333,1.0,1.0,1.0,0.0,1.0,0.666667,0.0,1.0


In [458]:
y = dataset.pop('price')
x = dataset.values
dataset.head()


,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,0.396564,0.6,0.333333,0.666667,1.0,0.0,0.0,0.0,1.0,0.666667,1.0,1.0
1,0.502405,0.6,1.000000,1.000000,1.0,0.0,0.0,0.0,1.0,1.000000,0.0,1.0
2,0.571134,0.4,0.333333,0.333333,1.0,0.0,1.0,0.0,0.0,0.666667,1.0,0.5
3,0.402062,0.6,0.333333,0.333333,1.0,0.0,1.0,0.0,1.0,1.000000,1.0,1.0
4,0.396564,0.6,0.000000,0.333333,1.0,1.0,1.0,0.0,1.0,0.666667,0.0,1.0


In [459]:
x = torch.tensor(x)
y = torch.tensor(y)

In [460]:
samples = x.shape[0]
n_val = int(.2 * samples)

shuffledindices = torch.randperm(samples)

trainindices = shuffledindices[:-n_val]
valindices = shuffledindices[-n_val:]

trainindices, valindices

(tensor([  5, 322, 268, 354, 339,  87, 112, 468, 310,  59,  81, 472, 344, 128,
         398, 146, 450, 537, 261,  83,  77, 126, 491, 527, 340, 502,   3, 367,
         326, 107, 465, 482, 334, 286,  64, 173,  41, 101,   0, 137, 544, 169,
         485, 350, 228,  51, 333, 431, 275, 314, 408, 193, 207, 258, 214, 337,
         200, 123, 363, 130, 241,  18, 375,   6, 486, 237, 323, 348,  91,  93,
         499,  12, 302, 172, 463, 336, 487, 428, 195,  78, 446, 176,  43, 360,
         283,  76, 422, 533, 187, 332,  57, 328, 358, 540, 493,  89, 404, 306,
         440, 266, 439, 117, 435, 342, 153,  50, 430, 229, 522, 299, 240, 205,
         401,  55, 211, 274, 165,  66, 389, 386, 361,  73, 458, 264, 372, 528,
          58, 462, 374, 277,  69, 543, 115, 420,  23,   8, 483, 243, 477, 409,
          42, 490, 279, 355, 170,  54, 476,  31, 366, 504, 212, 179, 394, 110,
         213, 149,  82, 234, 221, 405, 254, 414, 385,  46, 217,   9,  28, 495,
         196, 473,  61, 469, 474,  97,  75,  24, 368

In [461]:
xtrain = x[trainindices]
ytrain = y[trainindices]

valx = x[valindices]
valy = y[valindices]


In [462]:
xtrain = x[trainindices]
ytrain = y[trainindices]

valx = x[valindices]
valy = y[valindices]

xntrain = .1 * xtrain
xnval = .1 * valx

In [463]:
def training_loop(epochs, optimizer, model, loss_func, xtrain, ytrain, xtest, ytest):
  for epoch in range(1, epochs + 1):
    t_p_train = model(xtrain)
    loss_train = loss_func(t_p_train, ytrain)

    t_p_val = model(xtest)
    vloss = loss_func(t_p_val, ytest)

    optimizer.zero_grad()
    loss_train.backward()
    optimizer.step()


    if epoch % 500 == 0 :
      print(f"Epoch {epoch}, Training Loss {loss_train.item():.4f}, " f" Validation Loss {vloss.item():.4f}")



In [464]:
def loss_func(t_p, t_c):
  squared_diffs = (t_p - t_c) ** 2
  return squared_diffs.mean()

In [465]:
sequential_model = nn.Sequential(OrderedDict([
    ('hidden_linear', nn.Linear(12,32)),
    ('hidden_activation', nn.Tanh()),
    ('output_linear', nn.Linear(32, 1))
]))
sequential_model

Sequential(
  (hidden_linear): Linear(in_features=12, out_features=32, bias=True)
  (hidden_activation): Tanh()
  (output_linear): Linear(in_features=32, out_features=1, bias=True)
)

In [466]:
optimizer = optim.SGD(sequential_model.parameters(), lr = 1e-2)

training_loop(
    epochs = 3000,
    optimizer = optimizer,
    model = sequential_model,
    loss_func = nn.MSELoss(),
    xtrain = xntrain.to(torch.float32),
    ytrain = ytrain.to(torch.float32),
    xtest = xnval.to(torch.float32),
    ytest = valy.to(torch.float32)
)


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([436])) that is different to the input size (torch.Size([436, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.10/dist-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([109])) that is different to the input size (torch.Size([109, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Epoch 500, Training Loss 0.0275,  Validation Loss 0.0211
Epoch 1000, Training Loss 0.0275,  Validation Loss 0.0211
Epoch 1500, Training Loss 0.0275,  Validation Loss 0.0211
Epoch 2000, Training Loss 0.0275,  Validation Loss 0.0210
Epoch 2500, Training Loss 0.0275,  Validation Loss 0.0210
Epoch 3000, Training Loss 0.0275,  Validation Loss 0.0210
